In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [43]:
subprocess.run([
    'python', 'train_scripts/test_1110.py', 
    # '-s', 'saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/', 
    # '-s', 'saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VEF_finetune/1112_014010/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1112_013905/', 
    # '-s', 'saved/1113_Gosai_MPRA_MyResTransLateFusion_seq_Sei_VEF_finetune/1112_071608/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VDNase_finetune/1112_071248/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase_finetune/1112_071330/', 
    # '-s', 'saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase/1112_114300/', 
    # '-s', 'saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_164443/', 
    # '-s', 'saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/', 
    # '-s', 'saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_zscoreonrandom/1113_155631/', 
    # '-s', 'saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1114_054631',
    '-o', 'Gosai_pred.npy',
    '-de', 'cuda:0'],
    cwd='..'
)

use saved config: saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1114_054631/config.yaml


100%|██████████| 186/186 [01:03<00:00,  2.95it/s]


CompletedProcess(args=['python', 'train_scripts/test_1110.py', '-s', 'saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1114_054631', '-o', 'Gosai_pred.npy', '-de', 'cuda:0'], returncode=0)

In [2]:
MPRA_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_760679.tsv', sep='\t')
MPRA_df

,seq,ID,chr,pos,ref_allele,alt_allele,allele,OL,data_project,K562,HepG2,SK-N-SH,HCT116,A549
0,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:A:wC,chr1,14677,G,A,alt,OL41_42,GTEx,0.018,-0.048,0.017,-0.708,NaN
1,AGTGCTGGTTCCGTCACCCCCTCCCAAGGAAGTAGGTCTGAGCAGC...,1:14677:G:A:R:wC,chr1,14677,G,A,ref,OL41_42,GTEx,-1.043,0.191,0.099,-0.262,NaN
2,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:A:wC,chr1,63697,T,C,alt,OL41_42,GTEx,-1.005,0.650,-0.185,0.627,NaN
3,CATACATTTTTATTCTGGTCACTGTCCAACGACATTCCTCAAATGA...,1:63697:T:C:R:wC,chr1,63697,T,C,ref,OL41_42,GTEx,-0.779,0.762,0.281,0.676,NaN
4,AATAGACTCTAGTACTCTGTATTATGCAAAATTTGTCTATGTTACA...,1:64764:C:T:R:wC,chr1,64764,C,T,ref,OL41_42,GTEx,-0.455,-0.643,0.066,-0.164,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760674,CTGCATCTGTGCAGCCTGGCAGCGGCGGCGCTGCGCTGTGACATTT...,Y:15815629:NA:NA,chrY,15815629,NaN,NaN,NaN,OL15,OL15,0.716,1.309,1.089,NaN,NaN
760675,GCACTTCACATTTGTTTTTAGGGTTACATAGTCTACTCTGTATCCT...,Y:21740684:NA:NA,chrY,21740684,NaN,NaN,NaN,OL15,OL15,-0.708,-0.544,-0.427,NaN,NaN
760676,TGTTCAAGTGGCCACAGGGTTACTTGCTTTAGCATGGCTCCTTGGC...,Y:21740884:NA:NA,chrY,21740884,NaN,NaN,NaN,OL15,OL15,-0.143,-0.195,-0.308,NaN,NaN
760677,TAATTAGTTGGGAAGGTTCAGGTTCTGGGACATCCGTTGCTATTTC...,Y:21741084:NA:NA,chrY,21741084,NaN,NaN,NaN,OL15,OL15,-0.116,-0.084,-0.385,NaN,NaN


In [3]:
cell_types = ['K562', 'HepG2', 'SK-N-SH', 'HCT116', 'A549']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [4]:
data_split_dict = {}

data_split_dict['total'] = np.ones(len(MPRA_df), dtype=bool)
data_split_dict['train'] = MPRA_df['chr'].isin([f'chr{i}' for i in (list(range(3, 23)) + ['X', 'Y'])])
data_split_dict['valid'] = MPRA_df['chr'].isin(['chr1'])
data_split_dict['test']  = MPRA_df['chr'].isin(['chr2'])

# 和前三种细胞类型差异top5%的序列定义为cell type specific
for cell_type in cell_types:
    ref_mean = MPRA_df[cell_types[:3]].mean(axis=1)
    diff = (MPRA_df[cell_type] - ref_mean).abs()
    threshold = np.percentile(diff.dropna(), 99)
    data_split_dict[f'{cell_type}_specific'] = diff > threshold


for key in data_split_dict:
    print(key, data_split_dict[key].sum())

keys = list(data_split_dict.keys())
for k1 in keys:
    for k2 in keys:
        data_split_dict[f'{k1}+{k2}'] = data_split_dict[k1] & data_split_dict[k2]

total 760679
train 631073
valid 70952
test 58654
K562_specific 7607
HepG2_specific 7607
SK-N-SH_specific 7607
HCT116_specific 4482
A549_specific 3195


In [44]:
#### seq+VEF model
# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1112_013905/Gosai_pred.npy') # finetune版本,unseen上效果变差
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_AG_VEF_finetune/1112_014010/Gosai_pred.npy') # finetune版本,unseen上效果变差

y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1110_135744/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1111_Gosai_MPRA_MyResTrans_seq_AG_VEF/1110_135647/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1112_Gosai_MPRA_MyResTrans_seq_Sei_VDNase/1112_114300/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_164443/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1114_Gosai_MPRA_MyResTrans_seq_Sei_VEF/1112_173424/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_zscoreonrandom/1113_155631/Gosai_pred.npy')
# y_pred = np.load(f'../saved/1115_Gosai_MPRA_MyResTrans_seq_Sei_VEF_finetune/1114_054631/Gosai_pred.npy')

pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
MPRA_df[pred_cols] = y_pred





# #### seq model
# y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTrans_seq/1110_055401/Gosai_pred.npy')
# pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
# MPRA_df[pred_cols[:3]] = y_pred
# MPRA_df['HCT116_pred'] = y_pred.mean(axis=1)
# MPRA_df['A549_pred'] = y_pred.mean(axis=1)





# #### seq model (all cell types)
# y_pred = np.load(f'../saved/1110_Gosai_MPRA_MyResTrans_seq5/1112_093253/Gosai_pred.npy')
# pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
# MPRA_df[pred_cols] = y_pred
# MPRA_df.describe()




# #### VEF model
# y_pred = np.load(f'Gosai_MPRA_Sei_VEF_MLP_pred.npy')
# pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
# MPRA_df[pred_cols] = y_pred
# MPRA_df.describe()





# #### VEF alone
# # VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Sei_VEF_zscore.tsv', sep='\t')
# # VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Enformer_VEF_log1p.tsv', sep='\t')
# # VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_Borzoi_VEF_log1p.tsv', sep='\t')
# VEF_df = pd.read_csv('../data/Gosai_MPRA/Gosai_MPRA_AG_VEF_scalelog1p.tsv', sep='\t')
# DNase_cols = [f'{cell_type}_DNase' for cell_type in cell_types]
# pred_cols = [f'{cell_type}_pred' for cell_type in cell_types]
# MPRA_df[pred_cols] = VEF_df[DNase_cols]
# MPRA_df.describe()




MPRA_df[f'mean3'] = MPRA_df[cell_types[:3]].mean(axis=1)
MPRA_df[f'mean3_pred'] = MPRA_df[pred_cols[:3]].mean(axis=1)
MPRA_df.describe()

,pos,K562,HepG2,SK-N-SH,HCT116,A549,K562_pred,HepG2_pred,SK-N-SH_pred,HCT116_pred,A549_pred,mean3,mean3_pred
count,760679.000,760679.000,760679.000,760679.000,448103.000,319496.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000,760679.000
mean,75202958.598,-0.000,-0.000,0.000,0.000,-0.000,0.077,0.071,0.071,0.069,0.068,-0.000,0.073
std,57521411.054,1.000,1.000,1.000,1.000,1.000,1.018,0.999,1.003,1.014,1.005,0.930,0.983
min,14677.000,-5.360,-5.745,-5.762,-5.956,-6.017,-2.335,-2.700,-2.641,-2.643,-2.564,-4.647,-2.375
25%,28973267.000,-0.564,-0.597,-0.569,-0.567,-0.504,-0.503,-0.512,-0.504,-0.504,-0.507,-0.542,-0.496
50%,62363640.000,-0.283,-0.256,-0.206,-0.272,-0.203,-0.256,-0.246,-0.216,-0.257,-0.251,-0.250,-0.232
75%,112600466.000,0.206,0.284,0.285,0.208,0.215,0.233,0.267,0.268,0.232,0.234,0.225,0.262
max,249200985.000,7.854,7.705,8.315,7.761,11.771,7.840,7.776,7.799,7.819,7.834,7.583,7.805


In [45]:
# for data_split in ['total', 'test', 'test+specific']:
#     pearson_df = pd.DataFrame()
#     for c1 in cell_types:
#         for c2 in cell_types:
#             if data_split == 'specific':
#                 condition = data_split_dict[f'{c1}_specific']
#             elif data_split == 'test+specific':
#                 condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
#             else:
#                 condition = data_split_dict[data_split]
#             df = MPRA_df[condition]
#             x = df[f'{c1}']
#             y = df[f'{c2}_pred']
#             r, p = pearson(x, y)
#             pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
#     print(data_split, len(x))
#     print(pearson_df)






for data_split in ['total', 'test']:
    pearson_df = pd.DataFrame()
    for c1 in cell_types:
        for c2 in cell_types:
            if data_split == 'specific':
                condition = data_split_dict[f'{c1}_specific']
            elif data_split == 'test+specific':
                condition = data_split_dict[f'{c1}_specific'] & data_split_dict['test']
            else:
                condition = data_split_dict[data_split]
            df = MPRA_df[condition]
            x = df[f'{c1}'] - df['mean3']
            y = df[f'{c2}_pred'] - df['mean3_pred']
            r, p = pearson(x, y)
            # r, p = spearman(x, y)
            pearson_df.loc[f'{c1}', f'{c2}_pred'] = r
    print(data_split, len(x))
    print(pearson_df)

total 760679
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.665      -0.266        -0.477        0.187      0.064
HepG2       -0.289       0.555        -0.154       -0.055      0.049
SK-N-SH     -0.462      -0.207         0.654       -0.154     -0.113
HCT116      -0.012       0.064        -0.039        0.101      0.055
A549        -0.097      -0.066         0.160        0.110      0.146
test 58654
         K562_pred  HepG2_pred  SK-N-SH_pred  HCT116_pred  A549_pred
K562         0.540      -0.195        -0.374        0.152      0.029
HepG2       -0.219       0.466        -0.152       -0.034      0.077
SK-N-SH     -0.382      -0.214         0.541       -0.132     -0.101
HCT116       0.015       0.084        -0.079        0.091      0.040
A549        -0.088      -0.039         0.119        0.113      0.118
